In [2]:
import json
# %pip install requests # needed if running from a notebook and the library is not installed in the env
import requests
import argparse # For potential command-line arguments in a more advanced version

# --- Configuration ---
# These would typically be fetched from a config file, environment variables, or command-line arguments.
# For this assignment, they are hardcoded with the provided credentials.

DATABRICKS_WORKSPACE_URL = "https://dbc-28eb2f60-6b17.cloud.databricks.com/"
CLIENT_ID = "0294a4a7-054a-4210-8482-15c5d1aa1fe5" # Provided Service Principal Client ID
CLIENT_SECRET = "dosecbbb6cb467cdb795962e67738cf6a278"  # Provided Service Principal Secret

# Using the workspace-specific token endpoint and 'all-apis' scope as discovered
TOKEN_ENDPOINT = f"{DATABRICKS_WORKSPACE_URL.rstrip('/')}/oidc/v1/token"
OAUTH_SCOPE = "all-apis"

# Job ID(s) to validate permissions for.
# This would be provided by the customer or an external system.
# For this script, we use the Job ID discovered during development.
TARGET_JOB_IDS = ["10813307686450"]
EXPECTED_PERMISSION_LEVEL = "CAN_MANAGE"

# --- Core Functions ---

def get_databricks_access_token(token_url, client_id, client_secret, scope):
    """
    Obtains an OAuth M2M access token from Databricks.
    """
    payload = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': scope
    }
    try:
        response = requests.post(token_url, data=payload, timeout=20)
        response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
        token_data = response.json()
        if 'access_token' not in token_data:
            print(f"Error: 'access_token' not found in token response. Full response: {token_data}")
            return None
        return token_data['access_token']
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error obtaining token: {errh}")
        if errh.response is not None:
            print(f"Token Error Response: {errh.response.text}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception obtaining token: {err}")
    except json.JSONDecodeError as err_json:
        print(f"JSON Decode Error obtaining token: {err_json}. Response text: {errh.response.text if hasattr(errh, 'response') else 'N/A'}")
    return None

def get_job_permissions(workspace_url, access_token, job_id):
    """
    Retrieves the permissions for a specific Databricks job.
    """
    headers = {'Authorization': f'Bearer {access_token}'}
    api_url = f"{workspace_url.rstrip('/')}/api/2.0/permissions/jobs/{str(job_id)}"
    try:
        response = requests.get(api_url, headers=headers, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error getting permissions for Job ID {job_id}: {errh}")
        if errh.response is not None:
            if errh.response.status_code == 403:
                print(f"  Details: Access to Job ID {job_id} permissions is Forbidden. "
                      "The Service Principal may lack even CAN_VIEW permission on this job.")
            elif errh.response.status_code == 404:
                print(f"  Details: Job ID {job_id} not found.")
            else:
                print(f"  Response: {errh.response.text}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception getting permissions for Job ID {job_id}: {err}")
    except json.JSONDecodeError as err_json:
        print(f"JSON Decode Error for Job ID {job_id} permissions: {err_json}. Response text: {errh.response.text if hasattr(errh, 'response') else 'N/A'}")
    return None

def validate_job_permissions(job_id, permissions_data, principal_client_id, required_level):
    """
    Validates if the service principal has the required permission level on a job.
    """
    if not permissions_data or 'access_control_list' not in permissions_data:
        print(f"  Validation failed for Job ID {job_id}: Permissions data is missing or malformed.")
        return False

    for acl_entry in permissions_data.get('access_control_list', []):
        if acl_entry.get('service_principal_name') == principal_client_id:
            for permission in acl_entry.get('all_permissions', []):
                if permission.get('permission_level') == required_level:
                    print(f"  SUCCESS: Service Principal HAS '{required_level}' on Job ID {job_id}.")
                    return True
            print(f"  FAILURE: Service Principal FOUND for Job ID {job_id}, "
                  f"but does NOT have '{required_level}'. "
                  f"Found permissions: {acl_entry.get('all_permissions')}")
            return False

    print(f"  FAILURE: Service Principal NOT FOUND in ACL for Job ID {job_id}.")
    # print(f"Full ACL for Job ID {job_id}: {json.dumps(permissions_data.get('access_control_list'), indent=2)}") # Optional: for debugging
    return False

# --- Main Validation Logic ---

def run_permission_validation():
    """
    Main function to validate Databricks job permissions for the configured Service Principal.
    """
    print("--- Databricks Job Permission Validator ---")
    print(f"Workspace URL: {DATABRICKS_WORKSPACE_URL}")
    print(f"Service Principal Client ID: {CLIENT_ID}")
    print(f"Target Job IDs: {', '.join(TARGET_JOB_IDS) if TARGET_JOB_IDS else 'None'}")
    print(f"Expected Permission: {EXPECTED_PERMISSION_LEVEL}")
    print(f"Token Endpoint: {TOKEN_ENDPOINT} (Scope: {OAUTH_SCOPE})")
    print("-" * 50)

    if not CLIENT_ID or not CLIENT_SECRET:
        print("Error: Client ID or Client Secret is not configured.")
        return False

    access_token = get_databricks_access_token(TOKEN_ENDPOINT, CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE)
    if not access_token:
        print("Failed to obtain Databricks access token. Cannot proceed.")
        return False
    print("Successfully obtained Databricks access token.")
    print("-" * 50)

    if not TARGET_JOB_IDS:
        print("No target Job IDs specified. Nothing to validate.")
        return True # Or False, depending on if "nothing to do" is a success

    overall_success = True
    for job_id in TARGET_JOB_IDS:
        print(f"\nValidating permissions for Job ID: {job_id}...")
        permissions = get_job_permissions(DATABRICKS_WORKSPACE_URL, access_token, job_id)
        if permissions:
            if not validate_job_permissions(job_id, permissions, CLIENT_ID, EXPECTED_PERMISSION_LEVEL):
                overall_success = False
        else:
            print(f"  Could not retrieve or parse permissions for Job ID {job_id}. Validation failed for this job.")
            overall_success = False
        print("-" * 30)

    print("\n" + "=" * 50)
    if overall_success:
        print("OVERALL RESULT: SUCCESS! All specified jobs meet the required permission level.")
    else:
        print("OVERALL RESULT: FAILURE! Not all jobs meet the required permission level or errors occurred.")
    print("=" * 50)
    return overall_success

# --- Optional: Job Discovery Function ---

def discover_workspace_jobs(workspace_url, client_id, client_secret, token_url, scope):
    """
    Lists jobs accessible to the configured Service Principal.
    This function is for discovery and not part of the core validation flow by default.
    """
    print("\n--- Optional: Discovering Workspace Jobs ---")
    access_token = get_databricks_access_token(token_url, client_id, client_secret, scope)
    if not access_token:
        print("Failed to obtain access token for job discovery.")
        return

    print("Successfully obtained access token for job discovery.")
    headers = {'Authorization': f'Bearer {access_token}'}
    api_url = f"{workspace_url.rstrip('/')}/api/2.1/jobs/list"
    
    print(f"Attempting to list jobs from: {api_url}")
    try:
        response = requests.get(api_url, headers=headers, timeout=15)
        response.raise_for_status()
        jobs_data = response.json()

        if 'jobs' in jobs_data and jobs_data['jobs']:
            print("\nSuccessfully retrieved list of accessible jobs:")
            for job in jobs_data['jobs']:
                job_id_val = job.get('job_id')
                job_name = job.get('settings', {}).get('name', 'N/A')
                creator = job.get('creator_user_name', 'N/A')
                print(f"  - Job ID: {job_id_val:<15} Name: {job_name:<50} Creator: {creator}")
            print("\nNote: You can use these Job IDs to update TARGET_JOB_IDS for validation.")
        elif 'jobs' in jobs_data:
            print("No jobs found accessible to this Service Principal.")
        else:
            print(f"Warning: 'jobs' key not found in response. Full response: {json.dumps(jobs_data, indent=2)}")
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error listing jobs: {errh}")
        if errh.response is not None:
             print(f"  Response: {errh.response.text}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception listing jobs: {err}")
    except json.JSONDecodeError as err_json:
        print(f"JSON Decode Error listing jobs: {err_json}. Response text: {errh.response.text if hasattr(errh, 'response') else 'N/A'}")
    print("--- Job Discovery Finished ---")

# --- Script Execution ---

if __name__ == "__main__":
    # To run the main permission validation:
    run_permission_validation()

    # To discover jobs (optional, typically run once if Job IDs are unknown):
    # print("\n\nTo discover jobs, uncomment the line below and run the script again.")
    # discover_workspace_jobs(DATABRICKS_WORKSPACE_URL, CLIENT_ID, CLIENT_SECRET, TOKEN_ENDPOINT, OAUTH_SCOPE)

--- Databricks Job Permission Validator ---
Workspace URL: https://dbc-28eb2f60-6b17.cloud.databricks.com/
Service Principal Client ID: 0294a4a7-054a-4210-8482-15c5d1aa1fe5
Target Job IDs: 10813307686450
Expected Permission: CAN_MANAGE
Token Endpoint: https://dbc-28eb2f60-6b17.cloud.databricks.com/oidc/v1/token (Scope: all-apis)
--------------------------------------------------
Successfully obtained Databricks access token.
--------------------------------------------------

Validating permissions for Job ID: 10813307686450...
HTTP Error getting permissions for Job ID 10813307686450: 403 Client Error: Forbidden for url: https://dbc-28eb2f60-6b17.cloud.databricks.com/api/2.0/permissions/jobs/10813307686450
  Details: Access to Job ID 10813307686450 permissions is Forbidden. The Service Principal may lack even CAN_VIEW permission on this job.
  Could not retrieve or parse permissions for Job ID 10813307686450. Validation failed for this job.
------------------------------

OVERALL RESUL